# Задача: автоматизация ежедневного отчета по новостной ленте

## Цель

Настроить **ежедневную автоматическую отправку аналитического отчета** по новостной ленте в **Telegram** с помощью **Airflow**.

Отчет должен формироваться **каждое утро** и включать данные за **предыдущий день**, а также **динамику за последние 7 дней**.

---

## Что должно быть в отчете

Отчет состоит из двух частей:

### 1. **Текстовая часть**
Краткое описание ключевых метрик за **вчера**:

| Метрика | Описание |
|--------|----------|
| **DAU** | Ежедневная аудитория (Daily Active Users) |
| **Просмотры** | Общее число просмотров постов |
| **Лайки** | Общее число лайков |
| **CTR** | Click-Through Rate = Лайки / Просмотры |


### 2. **Графики**
Визуализация метрик за последние **7 дней**:

- График **DAU**
- График **CTR**
- График **просмотров и лайков** (можно в одной оси)

---

Файл в формате py

In [ ]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
import pandahouse as ph
from airflow.decorators import dag, task
from datetime import datetime, timedelta
from airflow.operators.python import get_current_context

my_token = '*************************' # тут нужно заменить на токен вашего бота
bot = telegram.Bot(token=my_token) # получаем доступ

connection = {
    'host': '*****',
    'password': '*****',
    'user': '*****',
    'database': '*****'
}

# Дефолтные параметры, которые прокидываются в таски
default_args = {
    'owner': 'm.kruzhalin',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2025, 6, 28)
}

#интервал выполнения дагов
schedule_interval = '0 11 * * *'

@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def maximkruzhalin_dag_tg_rep():
    
    @task()
    def extract_one():

        q = """select count(DISTINCT user_id) as DAU, countIf(action='view') as views, countIf(action='like') as likes, (countIf(action='like') / countIf(action='view')) as ctr from simulator_20250520.feed_actions where toDate(time) = today()-1"""

        # Получаем данные
        df_1 = ph.read_clickhouse(q, connection=connection)
        return df_1
        
    @task()
    def extract_two():
        
        qq ='''
            SELECT 
                toDate(time) AS day,
                count(DISTINCT user_id) AS DAU,
                countIf(action = 'view') AS views,
                countIf(action = 'like') AS likes,
                likes / views AS ctr
            FROM simulator_20250520.feed_actions 
            WHERE toDate(time) BETWEEN today() - 7 AND today() - 1
            GROUP BY day
            ORDER BY day
        '''

        df_2 = ph.read_clickhouse(qq, connection=connection)
        return df_2
        
    @task()
    def df_1_tg(df_1):

        unique_users = df_1['DAU'].iloc[0]
        views = df_1['views'].iloc[0]
        likes = df_1['likes'].iloc[0]
        ctr = df_1['ctr'].iloc[0]

        message = f"""
        📊 Отчет за вчера:
        👥 Уникальных пользователей: {unique_users}
        👁️ Просмотры: {views}
        👍 Лайки: {likes}
        📈 CTR: {ctr:.4f}
        """

        bot.sendMessage(chat_id=*****, text=message)
        
    @task()
    def df_2_tg(df_2):

        sns.set_style("whitegrid")
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        sns.lineplot(ax=axes[0, 0], x='day', y='DAU', data=df_2, marker='o', color='tab:blue')
        axes[0, 0].set_title('DAU — Уникальные пользователи')
        axes[0, 0].tick_params(axis='x', rotation=45)

        sns.lineplot(ax=axes[0, 1], x='day', y='views', data=df_2, marker='s', color='tab:orange')
        axes[0, 1].set_title('Просмотры (views)')
        axes[0, 1].tick_params(axis='x', rotation=45)

        sns.lineplot(ax=axes[1, 0], x='day', y='likes', data=df_2, marker='^', color='tab:green')
        axes[1, 0].set_title('Лайки (likes)')
        axes[1, 0].tick_params(axis='x', rotation=45)

        sns.lineplot(ax=axes[1, 1], x='day', y='ctr', data=df_2, marker='d', color='tab:red')
        axes[1, 1].set_title('CTR (likes / views)')
        axes[1, 1].tick_params(axis='x', rotation=45)

        plt.tight_layout()

        plot_object = io.BytesIO()
        plt.savefig(plot_object, format='png')
        plot_object.seek(0)
        plt.close()

        bot.sendPhoto(
            chat_id=*****,
            photo=plot_object,
            caption="📊 Ежедневный отчет за последние 7 дней"
        )
    df_1 = extract_one()
    df_2 = extract_two()
    
    df_1_tg(df_1)
    df_2_tg(df_2)

maximkruzhalin_dag_tg_rep =  maximkruzhalin_dag_tg_rep()

![tg_bot1](tg_bot1.png)